In [1]:
# Import the libraries
import numpy as np
import matplotlib.pyplot as plt  # for 畫圖用
import pandas as pd

In [2]:
dataset = pd.read_csv('./2330.TW.csv')  # 讀取訓練集

n = 1000
epochs_time = 50
rolling_time = 50
dataset_train = dataset.iloc[100:n]
dataset_test = dataset.iloc[n:]

In [3]:
training_set = dataset_train.iloc[:, 1:2].values  # 取「Open」欄位值
real_stock_price = dataset_test.iloc[:, 1:2].values

Reshape
因為現在 X_train 是 2-dimension，將它 reshape 成 3-dimension: [stock prices, timesteps, indicators]

input_shape是輸入資料格式，LSTM 層輸入格式: 為矩陣,矩陣內容 [ samples, time steps, features ] samples:觀測值，time steps:對於給定的觀測值,給定變數有單獨的時間步--就是時間視窗 features:在得到觀測值的時刻,觀測到的單獨的 measures--就是列數(屬性個數)

In [4]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

X_train = []   #預測點的前 rolling_time 天的資料
y_train = []   #預測點
for i in range(rolling_time, len(dataset_train)):
    X_train.append(training_set_scaled[i-rolling_time:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)  # 轉成numpy array的格式，以利輸入 RNN

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

搭建 LSTM layer:
units: 神經元的數目
第一層的 LSTM Layer 記得要設定input_shape參數
搭配使用dropout，這裡設為 0.2
由於這邊的第四層 LSTM Layer 即將跟 Ouput Layer 做連接，因此注意這邊的 return_sequences 設為預設值 False （也就是不用寫上 return_sequences）

In [ ]:
# Import the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

Using TensorFlow backend.
W0613 16:30:17.056380 140282034321216 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0613 16:30:17.071430 140282034321216 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0613 16:30:17.075146 140282034321216 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0613 16:30:17.254495 140282034321216 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:136: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instea

In [ ]:
# Adding the output layer
regressor.add(Dense(units = 1))

In [ ]:
# Compiling
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# 進行訓練
regressor.fit(X_train, y_train, epochs = epochs_time, batch_size = 32)

W0613 16:30:17.914035 140282034321216 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0613 16:30:18.207318 140282034321216 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0613 16:30:20.477326 140282034321216 deprecation_wrapper.py:119] From /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1021: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/50
850/850 [==============================] - 5s 5ms/step - loss: 0.0536
Epoch 2/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0091
Epoch 3/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0055
Epoch 4/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0044
Epoch 5/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0037
Epoch 6/50
850/850 [==============================] - 3s 3ms/step - loss: 0.0039
Epoch 7/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0036
Epoch 8/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0038
Epoch 9/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0034
Epoch 10/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0037
Epoch 11/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0034
Epoch 12/50
850/850 [==============================] - 2s 3ms/step - loss: 0.0037
Epoch 13/50
850/850 [====

In [ ]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - rolling_time:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs) # Feature Scaling

In [ ]:
X_test = []
for i in range(rolling_time, len(real_stock_price)+rolling_time):
    X_test.append(inputs[i-rolling_time:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))  # Reshape 成 3-dimension

In [ ]:
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)  # to get the original scale

In [ ]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real TSM Stock Price')  # 紅線表示真實股價
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted TSM Stock Price')  # 藍線表示預測股價
plt.title('TSM Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TSM Stock Price')
plt.legend()
plt.show()

In [ ]:
Predicted_future_list = []
for i in range(n):
    Predicted_future_list.append([np.nan])

for i in predicted_stock_price:
    Predicted_future_list.append(list(i))

In [ ]:
dataset_all = dataset.iloc[:, 1:2].values  

In [ ]:
inputs = dataset_all.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(rolling_time, len(dataset_all)):
    X_test.append(inputs[i-rolling_time:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_train_stock1_price = regressor.predict(X_test)
predicted_train_stock1_price = sc.inverse_transform(predicted_train_stock1_price)


In [ ]:
Predicted_before_list = []
for i in range(rolling_time):
    Predicted_before_list.append([np.nan])

for i in predicted_train_stock1_price:
    Predicted_before_list.append(list(i))

In [ ]:
plt.plot(dataset_all, color = 'red', label = 'Real TSM Stock Price')  # 紅線表示真實股價
plt.plot(Predicted_before_list, color = 'blue', label = 'Predicted before TSM Stock Price')  # 藍線表示預測股價
plt.plot(Predicted_future_list, color = 'green', label = 'Predicted future TSM Stock Price')  # 藍線表示預測股價


plt.title('TSM Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TSM Stock Price')
plt.legend()
plt.show()